In [1]:
%install '.package(url: "https://github.com/twostraws/SwiftGD.git", from: "2.0.0")' SwiftGD

Installing packages:
	.package(url: "https://github.com/twostraws/SwiftGD.git", from: "2.0.0")
		SwiftGD
Fetching https://github.com/twostraws/SwiftGD.git
Fetching https://github.com/twostraws/Cgd.git
Completed resolution in 1.97s
Cloning https://github.com/twostraws/Cgd.git
Resolving https://github.com/twostraws/Cgd.git at 0.2.0
Cloning https://github.com/twostraws/SwiftGD.git
Resolving https://github.com/twostraws/SwiftGD.git at 2.2.0
Compile Swift Module 'SwiftGD' (5 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Installation complete!

In [2]:
import Foundation

In [ ]:
import SwiftGD

: 